In [ ]:

import pandas as pd 
df = pd.read_csv('Churn_Modelling.csv')

numeric_columns_data = df.select_dtypes(include=('number'))
numeric_columns_data.corr()

y = df[['Exited']]
x = df[['Age','IsActiveMember','Balance']]

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)

import joblib 
joblib.dump(scaler, 'scaler2.pkl')

x_test = scaler.transform(x_test)

from sklearn.metrics import accuracy_score
def modelperformance(predictions):
    print(f'Accuracy Of My Model Is {accuracy_score(y_test, predictions)}')

from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression()
log_model.fit(x_train, y_train)
y_pred = log_model.predict(x_test)
modelperformance(y_pred)

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_neighbors': [2, 5, 7, 9],
    'weights': ['uniform', 'distance']
}
knn = KNeighborsClassifier()
gridkn = GridSearchCV(knn, param_grid, cv=5)
gridkn.fit(x_train, y_train)
print(gridkn.best_params_)
y_pred = gridkn.predict(x_test)
modelperformance(y_pred)

from sklearn.svm import SVC
from sklearn.calibration import CalibratedClassifierCV
param_grid = {
    'C': [0.01, 0.1, 0.5, 1],
    'kernel': ['linear', 'rbf', 'poly']
}
svc_base = SVC(class_weight='balanced', probability=True)
gridsvc = GridSearchCV(svc_base, param_grid, cv=5)
gridsvc.fit(x_train, y_train)
print(gridsvc.best_params_)
y_pred = gridsvc.predict(x_test)
modelperformance(y_pred)

from sklearn.tree import DecisionTreeClassifier
param_grid = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_tree = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=5)
grid_tree.fit(x_train, y_train)
print(grid_tree.best_params_)
y_pred = grid_tree.predict(x_test)
modelperformance(y_pred)

best_tree = grid_tree.best_estimator_
calibrated_model = CalibratedClassifierCV(best_tree, cv=5)
calibrated_model.fit(x_train, y_train)
joblib.dump(calibrated_model, 'model2.pkl')
y_pred = calibrated_model.predict(x_test)
modelperformance(y_pred)
